In [12]:
import warnings
warnings.filterwarnings(action='ignore')

import geopandas as gpd
import pandas as pd
import matplotlib as plt
import folium
import pyproj
from shapely.geometry import Point, Polygon, LineString
import geopy.distance

# CSV Upload

In [23]:
ddarung = pd.read_csv('../ddarung_master.csv', encoding = 'euc-kr')
ddarung = ddarung.drop(columns = ['Unnamed: 0'])
ddarung['public_transport_in_100m'] = ""
print(ddarung.shape)
ddarung.head(1)

(2719, 13)


,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,법정동,shortest_subway,shortest_subway_meter,public_place_in_100m,shortest_bus,shortest_bus_meter,public_transport_in_100m
0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,사직로12길,내자동,경복궁,183.7,0,서울경찰청.경복궁역,6.2,


In [7]:
bus_st = pd.read_csv('../ddarung_data/서울시 버스정류소 위치정보.csv', encoding = 'euc-kr')
print(bus_st.shape)
bus_st.head()

(12559, 4)


,정류소번호,정류소명,X좌표,Y좌표
0,1001,종로2가사거리,126.987750,37.569765
1,1002,창경궁.서울대학교병원,126.996566,37.579183
2,1003,명륜3가.성대입구,126.998340,37.582671
3,1004,종로2가.삼일교,126.987613,37.568579
4,1005,혜화동로터리.여운형활동터,127.001744,37.586243


In [11]:
bus_station = gpd.GeoDataFrame(
    bus_st, geometry=gpd.points_from_xy(bus_st.X좌표, bus_st.Y좌표), crs="EPSG:3310"
)
print(type(bus_station))
bus_station

<class 'geopandas.geodataframe.GeoDataFrame'>


,정류소번호,정류소명,X좌표,Y좌표,geometry
0,1001,종로2가사거리,126.987750,37.569765,POINT (126.988 37.570)
1,1002,창경궁.서울대학교병원,126.996566,37.579183,POINT (126.997 37.579)
2,1003,명륜3가.성대입구,126.998340,37.582671,POINT (126.998 37.583)
3,1004,종로2가.삼일교,126.987613,37.568579,POINT (126.988 37.569)
4,1005,혜화동로터리.여운형활동터,127.001744,37.586243,POINT (127.002 37.586)
...,...,...,...,...,...
12554,25995,우성아파트,127.139339,37.550386,POINT (127.139 37.550)
12555,25996,우성아파트,127.140046,37.550643,POINT (127.140 37.551)
12556,25997,조일약국,127.123596,37.533630,POINT (127.124 37.534)
12557,25998,성내시장,127.125497,37.536155,POINT (127.125 37.536)


In [9]:
subway_st = pd.read_csv('../ddarung_data/station_coordinate.csv')
print(subway_st.shape)
subway_st.head()

(730, 5)


,line,name,code,lat,lng
0,01호선,녹양,1908.0,37.759380,127.042292
1,01호선,남영,1002.0,37.541021,126.971300
2,01호선,용산,1003.0,37.529849,126.964561
3,01호선,노량진,1004.0,37.514219,126.942454
4,01호선,대방,1005.0,37.513342,126.926382


In [19]:
subway_station = gpd.GeoDataFrame(
    subway_st, geometry=gpd.points_from_xy(subway_st.lng, subway_st.lat), crs="EPSG:3310"
)
subway_station = subway_station.dropna()
subway_station

,line,name,code,lat,lng,geometry
0,01호선,녹양,1908.0,37.759380,127.042292,POINT (127.042 37.759)
1,01호선,남영,1002.0,37.541021,126.971300,POINT (126.971 37.541)
2,01호선,용산,1003.0,37.529849,126.964561,POINT (126.965 37.530)
3,01호선,노량진,1004.0,37.514219,126.942454,POINT (126.942 37.514)
4,01호선,대방,1005.0,37.513342,126.926382,POINT (126.926 37.513)
...,...,...,...,...,...,...
725,인천선,지식정보단지,3135.0,37.378384,126.645168,POINT (126.645 37.378)
726,인천선,인천대입구,3136.0,37.386007,126.639484,POINT (126.639 37.386)
727,인천선,센트럴파크,3137.0,37.393054,126.634729,POINT (126.635 37.393)
728,인천선,국제업무지구,3138.0,37.399907,126.630347,POINT (126.630 37.400)


# Publictransport Count in 100m

In [37]:
distance = 100 # meter

for index, row in ddarung.iterrows():
    transportcount = 0
    for i, row2 in subway_station.iterrows():#subway
        dist = geopy.distance.geodesic((row['위도'], row['경도']),(row2['lat'], row2['lng']))
        
        if(dist.meters <= distance):
            transportcount += 1
            
    for i, row2 in bus_station.iterrows():#bus
        dist = geopy.distance.geodesic((row['위도'], row['경도']),(row2['Y좌표'], row2['X좌표']))
        
        if(dist.meters <= distance):
            transportcount += 1
    
    print(index , " " , row['보관소(대여소)명'], " : " , transportcount)
    print(index , "/", ddarung.shape[0], " " , round(index / ddarung.shape[0] * 100, 2), "%")
    
    ddarung['public_transport_in_100m'][index] = transportcount
ddarung.head()

0    경복궁역 7번출구 앞  :  5
0 / 2719   0.0 %
1    경복궁역 4번출구 뒤  :  3
1 / 2719   0.04 %
2    광화문역 1번출구 앞  :  0
2 / 2719   0.07 %
3    종로구청 옆  :  1
3 / 2719   0.11 %
4    서울역사박물관 앞  :  4
4 / 2719   0.15 %
5    광화문 S타워 앞  :  3
5 / 2719   0.18 %
6    광화문역 6번출구 옆 B  :  7
6 / 2719   0.22 %
7    국립현대미술관  :  1
7 / 2719   0.26 %
8    종각역 1번출구 앞  :  5
8 / 2719   0.29 %
9    안국역 5번출구 앞  :  4
9 / 2719   0.33 %
10    낙원상가 옆  :  2
10 / 2719   0.37 %
11    탑골공원 앞  :  4
11 / 2719   0.4 %
12    청계2가 사거리 옆  :  6
12 / 2719   0.44 %
13    종로3가역 2번출구 뒤  :  3
13 / 2719   0.48 %
14    종로3가역 15번출구 앞  :  2
14 / 2719   0.51 %
15    창경궁 입구  :  5
15 / 2719   0.55 %
16    세운스퀘어 앞  :  3
16 / 2719   0.59 %
17    종로4가 사거리  :  3
17 / 2719   0.63 %
18    혜화동 로터리  :  4
18 / 2719   0.66 %
19    혜화역 3번출구 뒤  :  6
19 / 2719   0.7 %
20    대학로 마로니에공원  :  0
20 / 2719   0.74 %
21    예일빌딩(율곡로) 앞  :  1
21 / 2719   0.77 %
22    성균관대 E하우스 앞  :  2
22 / 2719   0.81 %
23    서울보증보험본사 앞  :  0
23 / 2719   0.85 %
24    독립문역 사거리  :  3
24 / 2719 

197    서빙고동 금호맨션 앞  :  4
197 / 2719   7.25 %
198    한남 유수지 복개주차장  :  2
198 / 2719   7.28 %
199    이태원지하보도  :  3
199 / 2719   7.32 %
200    녹사평역1번출구  :  4
200 / 2719   7.36 %
201    용산전쟁기념관  :  1
201 / 2719   7.39 %
202    삼각지역 3번출구  :  0
202 / 2719   7.43 %
203    LIG강촌아파트 103동앞  :  2
203 / 2719   7.47 %
204    신용산역 6번출구 앞  :  2
204 / 2719   7.5 %
205    삼각지역 4번출구 앞  :  0
205 / 2719   7.54 %
206    숙명여대 입구 교차로  :  2
206 / 2719   7.58 %
207    선린인터넷 고등학교  :  3
207 / 2719   7.61 %
208    청파동입구 교차로  :  2
208 / 2719   7.65 %
209    이촌1동 마을공원  :  2
209 / 2719   7.69 %
210    효창동주민센터 앞  :  2
210 / 2719   7.72 %
211    서빙고동 주민센터 앞  :  2
211 / 2719   7.76 %
212    국군복지단  :  0
212 / 2719   7.8 %
213    숙대입구역 8번  :  5
213 / 2719   7.83 %
214    베르가모앞  :  0
214 / 2719   7.87 %
215    청암자이아파트앞  :  4
215 / 2719   7.91 %
216    이태원관광특구입구  :  0
216 / 2719   7.94 %
217    이촌1동 주민센터 뒤  :  0
217 / 2719   7.98 %
218    효창공원앞역 3번출구 뒤  :  5
218 / 2719   8.02 %
219    남영역 건너편  :  5
219 / 2719   8.05 %
220  

382    중곡SK아파트앞  :  4
382 / 2719   14.05 %
383    구의아리수정수센터앞  :  4
383 / 2719   14.09 %
384    용마사거리  :  6
384 / 2719   14.12 %
385    군자역 7번출구뒤  :  4
385 / 2719   14.16 %
386    광진경찰서  :  3
386 / 2719   14.2 %
387    현대홈타운 뒷길  :  1
387 / 2719   14.23 %
388    건국대학교 과학관(이과대) 앞  :  0
388 / 2719   14.27 %
389    세종대학교  :  0
389 / 2719   14.31 %
390    광진정보도서관  :  2
390 / 2719   14.34 %
391    어린이대공원정문  :  3
391 / 2719   14.38 %
392    건대입구역 사거리(롯데백화점)  :  4
392 / 2719   14.42 %
393    건대입구역 5번출구 뒤  :  0
393 / 2719   14.45 %
394    중앙농협(자양동)  :  2
394 / 2719   14.49 %
395    아차산 휴먼시아 아파트 옆  :  2
395 / 2719   14.53 %
396    커먼그라운드  :  0
396 / 2719   14.56 %
397    래미안 구의파크 스위트  :  2
397 / 2719   14.6 %
398    어린이대공원역 3번출구 앞  :  2
398 / 2719   14.64 %
399    광진구의회 앞  :  0
399 / 2719   14.67 %
400    뚝섬유원지역 1번출구 앞  :  3
400 / 2719   14.71 %
401    자양사거리 광진아크로텔 앞  :  2
401 / 2719   14.75 %
402    광양중학교 앞  :  0
402 / 2719   14.78 %
403    광진메디칼 앞  :  1
403 / 2719   14.82 %
404    군자교교차로  :  0


568    홈플러스 신내점 앞  :  3
568 / 2719   20.89 %
569    면목2동 사거리  :  1
569 / 2719   20.93 %
570    신내능말공원  :  2
570 / 2719   20.96 %
571    중랑구 중소기업 창업센터  :  3
571 / 2719   21.0 %
572    면목4치안센터  :  2
572 / 2719   21.04 %
573    용마지구대  :  2
573 / 2719   21.07 %
574    중랑전화국 교차로  :  4
574 / 2719   21.11 %
575    면목역 3번출구  :  4
575 / 2719   21.15 %
576    코스트코 상봉점  :  3
576 / 2719   21.18 %
577    상봉역 1번출구  :  3
577 / 2719   21.22 %
578    화랑대역 7번출구  :  2
578 / 2719   21.26 %
579    중랑세무서  :  1
579 / 2719   21.29 %
580    겸재교 진입부  :  2
580 / 2719   21.33 %
581    중랑캠핑숲  :  1
581 / 2719   21.37 %
582    한국전력공사(동대문 중랑지사)  :  1
582 / 2719   21.4 %
583    상봉역 2번 출구  :  3
583 / 2719   21.44 %
584    상아빌딩(우림시장 교차로)  :  1
584 / 2719   21.48 %
585    동원사거리  :  2
585 / 2719   21.52 %
586    상봉터미널2  :  1
586 / 2719   21.55 %
587    용마한신아파트사거리  :  2
587 / 2719   21.59 %
588   동양엔파트 앞  :  3
588 / 2719   21.63 %
589   동부시장 북문 앞  :  2
589 / 2719   21.66 %
590   중랑구민회관  :  1
590 / 2719   21.7 %
591   중화역 4

753   서울북부고용노동지청  :  5
753 / 2719   27.69 %
754   번동사거리  :  4
754 / 2719   27.73 %
755   신창교  :  4
755 / 2719   27.77 %
756   강북문화정보도서관  :  5
756 / 2719   27.8 %
757    도봉세무서 앞  :  1
757 / 2719   27.84 %
758   4.19민주묘지역 1번출구  :  4
758 / 2719   27.88 %
759   수유사거리  :  2
759 / 2719   27.91 %
760   4.19민주묘지 입구사거리  :  1
760 / 2719   27.95 %
761   수유역2번출구  :  8
761 / 2719   27.99 %
762   북서울 꿈의 숲 입구  :  0
762 / 2719   28.03 %
763    우이동 먹거리마을  :  1
763 / 2719   28.06 %
764   현대성우아파트 옆  :  1
764 / 2719   28.1 %
765   송중초등학교 옆  :  2
765 / 2719   28.14 %
766   삼양입구삼거리  :  4
766 / 2719   28.17 %
767    드래곤타워 앞  :  2
767 / 2719   28.21 %
768    월계2교 교차로  :  2
768 / 2719   28.25 %
769    송천센트레빌  :  1
769 / 2719   28.28 %
770   번동3단지 버스정류장  :  3
770 / 2719   28.32 %
771    우이천 창번교  :  2
771 / 2719   28.36 %
772   삼양동주민센터 앞  :  2
772 / 2719   28.39 %
773   강북 청소년 문화정보도서관 앞  :  0
773 / 2719   28.43 %
774    녹천역 1번출구 앞  :  2
774 / 2719   28.47 %
775    도봉산광역환승센터앞  :  6
775 / 2719   28.5 %
776    도봉구청

...

2171   삼호3차아파트 앞  :  3
2171 / 2719   79.85 %
2172   멤피스아파트 앞  :  1
2172 / 2719   79.88 %
2173   국립국악원 앞  :  3
2173 / 2719   79.92 %
2174   한가람미술관 앞  :  3
2174 / 2719   79.96 %
2175   신반포역 3번출구 뒤  :  1
2175 / 2719   79.99 %
2176   서초종합체육관 입구  :  1
2176 / 2719   80.03 %
2177   양재역 8번출구 앞  :  4
2177 / 2719   80.07 %
2178   사평리 공원  :  3
2178 / 2719   80.1 %
2179   대한무역투자진흥공사 KOTRA 앞  :  1
2179 / 2719   80.14 %
2180   반포천 앞  :  3
2180 / 2719   80.18 %
2181   옛골 입구  :  2
2181 / 2719   80.21 %
2182   신논현타워 앞  :  1
2182 / 2719   80.25 %
2183   국립중앙도서관(국제회의장옆)  :  1
2183 / 2719   80.29 %
2184    반포래미안 리오센트  :  0
2184 / 2719   80.32 %
2185   반포1동 자율방범초소 옆  :  0
2185 / 2719   80.36 %
2186   LH 서초 4단지 403동  :  1
2186 / 2719   80.4 %
2187   양재동 꽃시장 건너편  :  0
2187 / 2719   80.43 %
2188    매헌역 2번출구앞광역버스 정류장앞(22-297)  :  1
2188 / 2719   80.47 %
2189    서울고앞  :  0
2189 / 2719   80.51 %
2190    서일스포츠센터  :  0
2190 / 2719   80.54 %
2191    서래공원 앞  :  1
2191 / 2719   80.58 %
2192    고투몰 gate 15옆  :  0
2192

2341    서울 논현동우체국 앞  :  1
2341 / 2719   86.1 %
2342    역삼중학교 앞(체육관 방향)  :  0
2342 / 2719   86.13 %
2343    서울율현초교 삼거리  :  1
2343 / 2719   86.17 %
2344    청담역 11번 출구  :  2
2344 / 2719   86.21 %
2345    휘문고교 사거리  :  1
2345 / 2719   86.24 %
2346    은마아파트입구 사거리(1)  :  0
2346 / 2719   86.28 %
2347    일원에코파크 앞(개포로 방면)  :  0
2347 / 2719   86.32 %
2348    대모산입구역 7번 출구 앞  :  0
2348 / 2719   86.36 %
2349    압구정나들목  :  0
2349 / 2719   86.39 %
2350    한국지역난방공사강남지사  :  0
2350 / 2719   86.43 %
2351    역삼1동 주민센터  :  0
2351 / 2719   86.47 %
2352    청담사거리(신한오피스텔빌딩 앞)  :  2
2352 / 2719   86.5 %
2353    개포우성6차아파트 정문 앞  :  1
2353 / 2719   86.54 %
2354    국악고교 앞 교차로  :  3
2354 / 2719   86.58 %
2355    삼성중앙역·플래티넘아파트 정류소  :  2
2355 / 2719   86.61 %
2356    세곡데이케어센터  :  1
2356 / 2719   86.65 %
2357    신청담나들목  :  1
2357 / 2719   86.69 %
2358    일원1동주민센터앞 사거리  :  2
2358 / 2719   86.72 %
2359    대청역  :  3
2359 / 2719   86.76 %
2360    LH강남힐스테이트아파트  :  0
2360 / 2719   86.8 %
2361    더시그넘하우스 앞  :  0
2361 / 2719   

2516    방이역 2번 출구  :  4
2516 / 2719   92.53 %
2517    올림픽공원 북2문  :  0
2517 / 2719   92.57 %
2518    건영아파트앞 사거리  :  2
2518 / 2719   92.61 %
2519    한림연예예술고등학교  :  2
2519 / 2719   92.64 %
2520    가락1동주민센터  :  1
2520 / 2719   92.68 %
2521    가락쌍용2차아파트 103동  :  2
2521 / 2719   92.72 %
2522    거여동 어울림아파트 102동앞  :  2
2522 / 2719   92.75 %
2523    송파파크데일 1단지 115동 건너편  :  0
2523 / 2719   92.79 %
2524    마천금호어울림 1차 102동앞  :  3
2524 / 2719   92.83 %
2525    가락현대 1차 11동 앞  :  0
2525 / 2719   92.87 %
2526    송파파인타운 4단지 403동  :  0
2526 / 2719   92.9 %
2527    문정근린공원2  :  1
2527 / 2719   92.94 %
2528    문정중학교  :  2
2528 / 2719   92.98 %
2529    가락시장 남1문 사거리  :  3
2529 / 2719   93.01 %
2530    올림픽공원 북2문 2  :  0
2530 / 2719   93.05 %
2531    잠실나들목4  :  0
2531 / 2719   93.09 %
2532    삼성SDS  :  1
2532 / 2719   93.12 %
2533    잠실나루역4번출구뒤  :  1
2533 / 2719   93.16 %
2534    잠실나루역 고가하부  :  0
2534 / 2719   93.2 %
2535    홈플러스 잠실점  :  0
2535 / 2719   93.23 %
2536    문현중고등학교 사이  :  2
2536 / 2719   93.27 %
25

2690    한영외국어고등학교  :  4
2690 / 2719   98.93 %
2691    강동리버스트4단지아파트 404동 앞  :  2
2691 / 2719   98.97 %
2692    강동리버스트4단지아파트 405동 앞  :  1
2692 / 2719   99.01 %
2693    성내삼성아파트  :  4
2693 / 2719   99.04 %
2694    둔촌동역교차로  :  4
2694 / 2719   99.08 %
2695    고덕중학교 후문  :  2
2695 / 2719   99.12 %
2696    강동리버스트6단지  :  3
2696 / 2719   99.15 %
2697    강동리버스트7단지  :  2
2697 / 2719   99.19 %
2698    강동리버스트8단지  :  2
2698 / 2719   99.23 %
2699    강동리엔파크9단지  :  0
2699 / 2719   99.26 %
2700    강동구음식물재활용센터  :  0
2700 / 2719   99.3 %
2701    강동역3번출구  :  2
2701 / 2719   99.34 %
2702    길동 마루빌딩  :  1
2702 / 2719   99.37 %
2703    길동 동호빌딩  :  0
2703 / 2719   99.41 %
2704    KT&G강동지사  :  4
2704 / 2719   99.45 %
2705    고덕자이(105동 앞)  :  3
2705 / 2719   99.49 %
2706    강동리엔파크14단지(1401동 앞)  :  0
2706 / 2719   99.52 %
2707    고덕 래미안 힐스테이트(101동)  :  4
2707 / 2719   99.56 %
2708    상일동산(동아아파트앞 교차로)  :  2
2708 / 2719   99.6 %
2709    강일역 3번출구  :  2
2709 / 2719   99.63 %
2710    강일역 4번출구  :  0
2710 / 2719   99.67 %

,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,법정동,shortest_subway,shortest_subway_meter,public_place_in_100m,shortest_bus,shortest_bus_meter,public_transport_in_100m
0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,사직로12길,내자동,경복궁,183.7,0,서울경찰청.경복궁역,6.2,5
1,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,사직로,적선동,경복궁,51.1,0,경복궁역(효자로입구),76.2,3
2,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.571770,126.974663,세종대로23길,당주동,광화문,195.5,0,광화문,161.8,0
3,종로구청 옆,종로구,서울특별시 종로구 세종로 84-1,37.572559,126.978332,종로1길,세종로,광화문,224.9,0,KT광화문지사,95.8,1
4,서울역사박물관 앞,종로구,서울특별시 종로구 새문안로 55 서울역사박물관 앞,37.570000,126.971100,경희궁1길,신문로2가,광화문,505.0,3,서울역사박물관.경교장.강북삼성병원,13.3,4


In [38]:
ddarung.to_csv('../ddarung_master.csv', encoding = 'euc-kr')